# Validate EIA-930 data against net generation outputs

In [2]:
import pandas as pd
import numpy as np
import os

import plotly.express as px

In [3]:
year = 2020

In [4]:
# EIA-930 data after timestamp adjustments but no cleaning
raw = pd.read_csv(f"../data/outputs/{year}/eia930/eia930_raw.csv", index_col=0, parse_dates=True)

In [14]:
GEN_ID = "EBA.{}-ALL.NG.H"
path = f"../data/results/{year}/power_sector_data/hourly/us_units/"
cors = {}
percent_difs = {}
annual_gen = {}
for ba_f in os.listdir(path):
    ba = ba_f.replace(".csv", "")
    print(ba, end="...")
    col_name = GEN_ID.format(ba)
    if col_name not in raw.columns: 
        continue
    else:
        dat = pd.read_csv(path+ba_f, parse_dates=["datetime_utc"])
        dat = dat[dat.fuel_category=="total"]
        dat = dat.merge(raw[ col_name], left_on="datetime_utc", right_index=True)
        c = dat[["net_generation_mwh", col_name]].corr().to_numpy()[0,1]
        cors[ba] = c
        difs = (dat[col_name]-dat["net_generation_mwh"])/dat["net_generation_mwh"]
        difs = difs.replace(np.inf, np.nan)
        percent_difs[ba] = difs.median()
        annual_gen[ba] = dat["net_generation_mwh"].sum()

OHMS...FMPP...ERCO...TPWR...AMPL...SOCO...IPCO...PJM...WWA...EEI...AZPS...TEC...DOPD...YAD...IID...HGMA...DEAA...CPLW...SPA...GVL...ORMS...FPL...TAL...SEC...JEA...GCPD...TVA...TIDC...HECO...SCEG...INMS...CPLE...SEPA...AKMS...CHPD...PNM...WAUW...WACM...WALC...NWMT...PACE...AVA...SC...NSB...GWA...HIMS...LGEE...TEPC...ISNE...SRP...GRIS...HST...LDWP...RIMS...FPC...PACW...GRIF...PSEI...AECI...CSTO...DUK...AVRN...CEA...MISO...AEC...PSCO...OVEC...SWPP...BANC...NYIS...EPE...NBSO...SCL...BPAT...NEVP...CISO...PGE...

In [5]:
out = pd.DataFrame(data={"Difference as percent of hourly-egrid":percent_difs, "Correlation":cors, "Annual BA generation":annual_gen})
out = out.sort_values("Annual BA generation", ascending=False)
out.to_csv(f"../data/results/{year}/validation_metrics/us_units/compare_930_hourlyegrid.csv")

NameError: name 'percent_difs' is not defined

# Visualize BA of interest

In [9]:
ba = "SWPP"
col_name = GEN_ID.format(ba)
dat = pd.read_csv(path+ba+".csv", parse_dates=["datetime_utc"])
dat = dat[dat.fuel_category=="total"]
dat = dat.merge(raw[ col_name], left_on="datetime_utc", right_index=True)

px.line(dat, x="datetime_utc", y=["net_generation_mwh", col_name])

In [23]:
residual_profiles = pd.read_csv(f"../data/outputs/{year}/residual_profiles_{year}.csv", parse_dates=["datetime_utc"])

In [24]:
toplot = residual_profiles[(residual_profiles.ba_code==ba) & (residual_profiles.fuel_category_eia930 =="natural_gas")]
px.line(toplot,x="datetime_utc", y="profile")

# Scrape Singularity emission and rate data 

* Drawbacks: 
    * won't have timestamp fix 
    * slow the first time (cache locally)

* Benefits: 
    * Don't have to re-do emission logic 